### Group C
- Campos, Joshua
- Halili, Gesara
- Nwuzor, Chisom
- Tran, Quynh

# Data Science Project 2020

In this project, we will be tackling a supervised classification problem using the data from the Kaggle competition "[Two Sigma Connect: Rental Listing Inquiries](https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries)". Our task will be to classify rental listing inquiries, and predict how much interest they will receive according to three categories: Low, Medium and High. 

For this project, we will be using three different models, and comparing their performance to find the best one for this specific task. The models will be:
- Regularized Linear Models
- Trees
- Random Forests
- Neural Networks

Our classification problem will be composed of various sections: 
- Preparation & Exploratory Data Analysis
- Data Preprocessing & Feature Engineering
- Model Performance & Hyper-Parameter Tuning
- Prediction Explanation & Story Telling

## Preparation & Exploratory Data Analysis

### Importing the libraries

The first thing we have to do is to import all the libraries that we will be using for our project, which include the typical libraries for data science, such as Numpy, Pandas, Matplotlib, and Scikit-Learn, among others.

In [ ]:
! pip install keras

In [ ]:
import json
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string 

from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Dropout

from pylab import rcParams
from re import search 

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, LabelEncoder, OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier

from pdpbox import pdp, get_dataset, info_plots
from sklearn.inspection import permutation_importance
from time import time
from rfpimp import *
import shap

import functions

### Reading the file

Here we read the data, which is stored as a JSON file. We will be only using the training data for the whole project, which we will split into training and testing sets respectively. This is done to decrease the processing power and time required for the model training and the hyper-parameter tuning.

In [ ]:
with open('data/train.json') as f:
    data = json.load(f)

df = pd.DataFrame(data)

### Exploratory Data Analysis

Before continuing, we must get a grasp of what the data looks like. For this, we print the head of the data, as well as some statistics and information about the variables. Besides this, we also take a look at the unique number of values for the display addresses, the manager ids, and the features. By doing this, we understand if it would be wise to treat this features as categorical or not. 

In [ ]:
df_head = df.head()
df_describe = df.describe()
df_info = df.info()

print('\nNum. of Unique Display Addresses: {}'.format(df['display_address'].nunique()))
print('Num. of Unique Manager IDs: {}'.format(df['manager_id'].nunique()))

all_unique_features = set()    
df['features'].apply(lambda x: functions.add_unique_elements(x, all_unique_features))
print('Num. of Unique Features: {}'.format(len(all_unique_features)))

We also split the variables into some of their different data types for easier handling later on. And finally, we encode the target variable from text to numerical values, ranging from 0 to 2, and plot the counts for each class to get an idea of how balanced or unbalanced the dataset is.

In [ ]:
num_vars = ['bathrooms','bedrooms','latitude','longitude','price']
cat_vars = ['building_id','manager_id','display_address','street_address']
text_vars = ['description','features']

rcParams['figure.figsize'] = 11.7,8.27
df['interest_level'] = df['interest_level'].apply(lambda x: 0 if x=='low' else 1 if x=='medium' else 2)
# Check homogeneity of target values
sns.countplot('interest_level', data=df)
plt.title('Unbalanced Classes')
plt.show()

## Data Preprocessing & Feature Engineering

### Feature Engineering

After taking a look at the current features available in the dataset, we notice that there are some variables with which we cannot work just yet. So as a first step, before going into the modeling journey, we decide to do some feature engineering in order to extract some more value from the existing features and create new variables that might help us in the next steps. 

For this section, we used, as a reference, the notebook available in Kaggle called "[TwoSigmaRenthop - Advanced Feature Engineering](https://www.kaggle.com/chriscc/twosigmarenthop-advanced-feature-engineering)". This notebook helped us understand different ways to handle several of the variables present in the notebook to add more value to our project.

#### Categorical Variables

In [ ]:
addresses = ['display_address', 'street_address']

for address in addresses:
    print(address)
    ''' delete rows that contain descriptions instead of actual addresses '''
    address_delete = [] 
    for i in range(len(df)):
        address_val = df[address][i]
        if search('!' or '*', address_val):
            address_delete.append(i)

    df = df.drop(df.index[address_delete])
    print("Num. of deleted addresses: {}".format(
        len(address_delete)))
    
    
    ''' Data Cleaning '''
    address_column = df[address]
    address_column_transformed = ( address_column
                                           .apply(str.upper)
                                           .apply(lambda x: x.replace('WEST','W'))
                                           .apply(lambda x: x.replace('EAST','E'))
                                           .apply(lambda x: x.replace('STREET','ST'))
                                           .apply(lambda x: x.replace('AVENUE','AVE'))
                                           .apply(lambda x: x.replace('BOULEVARD','BLVD'))
                                           .apply(lambda x: x.replace('.',''))
                                           .apply(lambda x: x.replace(',',''))
                                           .apply(lambda x: x.replace('&',''))
                                           .apply(lambda x: x.replace('(',''))
                                           .apply(lambda x: x.replace(')',''))
                                           .apply(lambda x: x.strip())
                                           #.apply(lambda x: re.sub('(?<=\d)[A-Z]{2}', '', x))
                                           .apply(lambda x: re.sub('[^A-Za-z0-9]+ ', '', x)) #remove all special characters and punctuaction
                                           .apply(lambda x: x.replace('FIRST','1'))
                                           .apply(lambda x: x.replace('SECOND','2'))
                                           .apply(lambda x: x.replace('THIRD','3'))
                                           .apply(lambda x: x.replace('FOURTH','4'))
                                           .apply(lambda x: x.replace('FIFTH','5'))
                                           .apply(lambda x: x.replace('SIXTH','6'))
                                           .apply(lambda x: x.replace('SEVENTH','7'))
                                           .apply(lambda x: x.replace('EIGHTH','8'))
                                           .apply(lambda x: x.replace('EIGTH','8'))
                                           .apply(lambda x: x.replace('NINTH','9'))
                                           .apply(lambda x: x.replace('TENTH','10'))
                                           .apply(lambda x: x.replace('ELEVENTH','11'))
                                         )

    print("Num. of Unique Addresses after Transformation: {}".format(
        address_column_transformed.nunique()))

    df[address] = address_column_transformed 

#Count features
display=df["display_address"].value_counts()
manager_id=df["manager_id"].value_counts()
building_id=df["building_id"].value_counts()
street=df["street_address"].value_counts()

df["display_count"]=df["display_address"].apply(lambda x:display[x])
df["manager_count"]=df["manager_id"].apply(lambda x:manager_id[x])  
df["building_count"]=df["building_id"].apply(lambda x:building_id[x])
df["street_count"]=df["street_address"].apply(lambda x:street[x])

#Numeric-categorical interactions (price and building)
price_by_building = df.groupby('building_id')['price'].agg([np.min,np.max,np.mean]).reset_index()
price_by_building.columns = ['building_id','min_price_by_building',
                            'max_price_by_building','mean_price_by_building']
df = pd.merge(df,price_by_building, how='left',on='building_id')
df = df.drop(df.index[address_delete])

OE = OrdinalEncoder()
for cat_var in cat_vars:
    print ("Ordinal Encoding %s" % (cat_var))
    df[cat_var]=OE.fit_transform(df[[cat_var]])

#### Text Variables

In [ ]:
'''
Count number of special characters in description:
Studies have shown that titles with excessive all caps and special characters give renters the impression 
that the listing is fraudulent – i.e. BEAUTIFUL***APARTMENT***CHELSEA.
'''

df['num_of_#']=df.description.apply(lambda x:x.count('#'))
df['num_of_!']=df.description.apply(lambda x:x.count('!'))
df['num_of_$']=df.description.apply(lambda x:x.count('$'))
df['num_of_*']=df.description.apply(lambda x:x.count('*'))
df['num_of_>']=df.description.apply(lambda x:x.count('>'))

#Derive new features (check for phone and email) out of description 
df['has_phone'] = df['description'].apply(lambda x:re.sub('['+string.punctuation+']', '', x).split())\
        .apply(lambda x: [s for s in x if s.isdigit()])\
        .apply(lambda x: len([s for s in x if len(str(s))==10]))\
        .apply(lambda x: 1 if x>0 else 0)

print('Has phone: ' + str(np.round(len(np.where(df['has_phone'] == 1)[0]) / len(df), 2)) + '%')
print('Has no phone: '+ str(np.round(len(np.where(df['has_phone'] == 0)[0]) / len(df), 2)) + '%')

df['has_email'] = df['description'].apply(lambda x: 1 if '@renthop.com' in x else 0)
print('Has email: '+ str(np.round(len(np.where(df['has_email'] == 1)[0]) / len(df), 2))+ '%')
print('Has no email: '+ str(np.round(len(np.where(df['has_email'] == 0)[0]) / len(df), 2))+ '%')

# Count length, num of words and features of description of features 
display_address_column = df['description']
df['description'] = [functions.text_cleaner(x) for x in display_address_column]

df['length_description'] = df['description'].apply(lambda x: len(x))
df['num_words_description'] = df['description'].apply(lambda x: len(x.split(" ")))

df['num_features'] = df['features'].apply(len)

#Extract tokens of features and OHE 
v = CountVectorizer(stop_words='english', max_features=50)
x = v.fit_transform(df['features']\
                                     .apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x])))

df1 = pd.DataFrame(x.toarray(), columns=v.get_feature_names())
df.drop('features', axis=1, inplace=True)
df = df.join(df1.set_index(df.index))

tokens_features = v.get_feature_names()
print(tokens_features)

#### Date Variables

In the dataset, we have one text variable that refers to the date the rental advertisement was published. In order to use this variable we have to convert it from text to actual datetime type. After we do this, we decide to get some more features from this variable, including the year, month, day of month, day of week, and hour of publication. 

In [ ]:
df['created'] = pd.to_datetime(df['created'])
df['created_year'] = df['created'].dt.year
df['created_month'] = df['created'].dt.month
df['created_day_of_month'] = df['created'].dt.day
df['created_day_of_week'] = df['created'].dt.dayofweek
df['created_hour'] = df['created'].dt.hour

#### Image Variables

In [ ]:
df['num_photos'] = df['photos'].apply(len)
df['photos_per_bedroom'] = df[['num_photos','bedrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)
df['photos_per_bathroom'] = df[['num_photos','bathrooms']].apply(lambda x: x[0]/x[1] if x[1]!=0 else 0, axis=1)

#### Numerical Variables

From the previous section, we noticed that we have several numerical variables, and after many of the transformations, we end up with even more of them. Becase of this, we decide to create some logical interactions that might have an impact on our model. 

We decide to calculate the total numer of rooms, which is the bedrooms plus the bathrooms. We then decide to calculate the price per room, bedroom and bathroom, where we divide the price by the number of the previously mentioned values. 

In [ ]:
df['total_rooms'] = df['bathrooms'] + df['bedrooms']
df['price_per_room'] = df[['price','total_rooms']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)
df['price_per_bedroom'] = df[['price','bedrooms']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)
df['price_per_bathroom'] = df[['price','bathrooms']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)

We also decide to do something similar like before, but this time we calculate the price per word in the word description, as well as the price per length of the description. Moreover, we do the same thing with the features and photos, by calculating the price by each of them. 

In [ ]:
df['price_per_word_description'] = df[['price','num_words_description']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)
df['price_per_length_description'] = df[['price','length_description']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)
df['price_per_feature'] = df[['price','num_features']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)
df['price_per_photo'] = df[['price','num_photos']].apply(lambda x: x[0]/x[1] if x[1] != 0 else 0, axis=1)

Finally, we also notice that we have the location of the rented properties. We know that the location usually has an impact on the real estate, so we decide to calculate the distances to some well-known areas of New York City, including the Central Park, Wall Street, and Times Square. In order to calculate the distances, we use a function that we previously defined, that takes the differences of the coordinates and converts the difference into kilometers, so it makes more sense to us. 

In [ ]:
central_park_coordinates = (40.7799963,-73.970621)
df['distance_to_central_park'] = df[['latitude','longitude']].apply(
        lambda x: functions.calculate_distance_between_coordinates(central_park_coordinates,(x[0],x[1])), axis=1)

wall_street_coordinates = (40.7059692,-74.0099558)
df['distance_to_wall_street'] = df[['latitude','longitude']].apply(
        lambda x: functions.calculate_distance_between_coordinates(wall_street_coordinates,(x[0],x[1])), axis=1)

times_square_coordinates = (40.7567473,-73.9888876)
df['distance_to_times_square'] = df[['latitude','longitude']].apply(
        lambda x: functions.calculate_distance_between_coordinates(times_square_coordinates,(x[0],x[1])), axis=1)

### Correlation of Features and Target

In [ ]:
""" Object columns dropped"""
df = df.drop(['building_id', 'listing_id', 'description', 'created', 'created_year','display_address', 'manager_id', 
              'photos', 'street_address'], axis=1) 

# Convert target values into ordinal values 

df_corr = df.corr()
df_corr_abs = np.abs(df_corr['interest_level'])

df_corr_abs_sort = df_corr_abs.sort_values(ascending = False)
print(df_corr_abs_sort)

sns.set(rc={'figure.figsize':(15.7,10.27)})
sns.heatmap(df.corr())

## Model Performance & Hyper-Parameter Tuning

### Splitting of the Dataset

After having performed the feature engineering and decided that our dataset is ready to be modeled, we first have to split our dataset into training and testing set. For this case, we first separate our independent variables (X) from our target variable (y). Since we are going to be using cross-validation, we decide to split only into training and testing set, without manually splitting also into a validation set, which will be created automatically during the cross-validation. As a testing size, we decide to use 15% of the data. 

In [ ]:
X = df.drop("interest_level", axis=1)
y = df["interest_level"]

X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.15)

### Feature Selection

In [ ]:
sel = SelectFromModel(RandomForestClassifier())
sel.fit(X_train, y_train)


#To see which feature are important
sel.get_support()


#to get the number of important features
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)


#to get the name of feature selected
print(selected_feat)


X_train= X_train[selected_feat]
X_test= X_test[selected_feat]


### Hyperparameter Tuning  for the Random Forest

In [ ]:
 # Number of trees in random forest
n_estimators = np.arange(start=100, stop=2001, step=10)
 #Number of features to consider at every split'''
max_features = ['auto', 'sqrt']
 #Maximum number of levels in tree'''
max_depth = np.arange(start=10, stop=111, step=5)
#max_depth.append(None)
 #Minimum number of samples required to split a node'''
min_samples_split = np.arange(start=2, stop=101)
 #Minimum number of samples required at each leaf node'''
min_samples_leaf = np.arange(start=2, stop=101)
#Method of selecting samples for training each tree'''
bootstrap = [True, False]


'''Create the random grid'''
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


'''First create the base model to tune'''

rf = RandomForestClassifier()

''' Use the random grid to search for best hyperparameters'''
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 5, verbose=2, 
                               random_state=123, n_jobs = -1)

'''Fit the random search model'''
rf_random.fit(X_train, y_train)

print('test')

best_params = rf_random.best_params_

print(best_params)

### Pruning of the Decision Tree

In [ ]:
tree = DecisionTreeClassifier(criterion = "gini", random_state = 123) 
tree = tree.fit(X_train, y_train) 
y_pred = tree.predict(X_test) 
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 



#ploting the tree

from sklearn.tree import plot_tree
plt.figure(figsize = (15,17))
plot_tree(tree, filled = True, rounded = True, class_names = ['Low Interest', 'Medium Interest', 'High Interest'], feature_names = X.columns)[0]

#determine values for alpha
path = tree.cost_complexity_pruning_path(X_train, y_train)

#extract different values for alpha
ccp_alphas = path.ccp_alphas 
ccp_alphas = ccp_alphas[:-1]


#create an array to store the resuls of each fold during cross validation
alpha_loop_values = []

# For each candidate value alpha, we will run 5-fold cross validation
# Then we will store the mean and std of the scores(accuracy) for each call
# to cross_val_score in alpha_loop_values...

for ccp_alpha in ccp_alphas:
    tree = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    scores = cross_val_score(tree, X_train, y_train, cv=5)
    alpha_loop_values.append([ccp_alpha, np.mean(scores), np.std(scores)])
#Now we can draw a graph of the means and std of the scores
#for each candidate value for alpha
alpha_results = pd.DataFrame(alpha_loop_values, 
                             columns=['alpha', 'mean_accuracy', 'std'])

alpha_results.plot(x='alpha',
                   y='mean_accuracy',
                   yerr = 'std',
                   marker = 'o',
                  linestyle ='--')


#Printing the best alpha

column = alpha_results["mean_accuracy"]
max_index = column.idxmax()
max_index


ideal_ccp_alpha = alpha_results.iloc[max_index]['alpha']
#ideal_ccp_alpha 
#Prunning the classification tree

tree_pruned = DecisionTreeClassifier(random_state = 123,
                                    ccp_alpha=ideal_ccp_alpha)
tree_pruned = tree_pruned.fit(X_train, y_train)




y_pred = tree_pruned.predict(X_test) 
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 

#plotting the best tree
plt.figure(figsize = (20,17))
plot_tree(tree_pruned, filled = True, rounded = True,class_names = ['Low Interest', 'Medium Interest', 'High Interest'], feature_names = X.columns)[0]

### Model Selection

In [ ]:
#Defining model parameters from the tuned parameter
model_params = {
    'random_forest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': best_params['n_estimators'],
            'min_samples_split': best_params['min_samples_split'],
            'min_samples_leaf': best_params['min_samples_leaf'],
            'max_features': best_params['max_features'],
            'max_depth': best_params['max_depth'],
            'bootstrap': best_params['bootstrap']
        }
    },
    
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'ccp_alpha': ideal_ccp_alpha
        }
    },
    
}

#Loop over both decision tree and random forest to determine the best model for the given dataset
scores = []
print(model_params.items())

trained_models = []
for model_name, mp in model_params.items():
    clf = mp['model']
    clf.set_params(**mp['params'])
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    scores.append({
        'model': model_name,
        'train_score': clf.score(X_train, y_train),
        'test_score': clf.score(X_test, y_test),
        'log_loss': log_loss(y_test, y_pred)
    })
    trained_models.append(clf)
    
best_model = pd.DataFrame(scores, columns= ['model','train_score','test_score', 'log_loss'])
print(best_model)

rf = trained_models[0]

## Prediction Explanation & Story Telling

### Feature Importance

In [ ]:
#MDI
MDI_importances = rf.feature_importances_
indices = np.argsort(MDI_importances)
features = X_train.columns

#MDA
MDA_test = permutation_importance(rf, X_test, y_test, n_repeats=10, random_state=123)
sorted_idx1 = MDA_test.importances_mean.argsort()
MDA_train = permutation_importance(rf, X_train, y_train, n_repeats=10, random_state=123)
sorted_idx2 = MDA_train.importances_mean.argsort()

#Plotting
plt.rcParams["figure.figsize"]=15,5
fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.set_title('MDI Importances')
ax1.barh(range(len(indices)), MDI_importances[indices], color='b', align='center')
ax1.set_yticks( np.arange(len(X_train.columns)))
ax1.set_yticklabels(features[indices])
ax1.set(xlabel='Relative Importance')
ax2.boxplot(MDA_train.importances[sorted_idx2].T,
vert=False, labels=features[sorted_idx2])
tpmp=ax2.set_title("Permutation Importances (train)")
ax3.boxplot(MDA_test.importances[sorted_idx1].T,
vert=False, labels=features[sorted_idx1])
tpmp=ax3.set_title("Permutation Importances (test)")
fig.tight_layout()

#Table of results
# record times for comparison
t0 = time()
# Gini importance
gini_imp = pd.DataFrame({'Feature': X_train.columns, 'Gini Importance': rf.feature_importances_}).set_index('Feature')

t1 = time()# Permutation importance for train
perm_imp = importances(rf, X_train, y_train)

t2 = time()
res= gini_imp.merge(perm_imp, left_index=True, right_index=True).reset_index().\
        rename(columns={'Importance': 'Permutation Importance'})
res.loc[len(X_train.columns)+1] = ['runtime(s)', t1-t0, t2-t1]

print(res)

top5_feat = res[:-1].sort_values(by=['Permutation Importance'], ascending=False).head(5)
print('Top 5 features with higher importance:', top5_feat)

Explain what pdp-s are

In [ ]:
ploting_pdp(f=top5_feat.iloc[0,0])

explain result 1

In [ ]:
ploting_pdp(f=top5_feat.iloc[1,0])

explain result 2

In [ ]:
ploting_pdp(f=top5_feat.iloc[2,0])

explain result 3

In [ ]:
ploting_pdp(f=top5_feat.iloc[3,0])

explain result 4

In [ ]:
ploting_pdp(f=top5_feat.iloc[4,0])

explain result 5

Explain nex steps

In [ ]:
get_top_abs_correlations(df, n=5)

Explain 2 dimension pdp-s

In [ ]:
#Plot 2 dimension pdp for top 5 correlations
features_to_plot = ['price', 'bedrooms'] #manually add the top 5 features
two_dim_pdp(f=features_to_plot)

Exlain result 1

In [ ]:
#Plot 2 dimension pdp for top 5 correlations
features_to_plot = ['price', 'bedrooms']
two_dim_pdp(f=features_to_plot)

Exlain result 2

In [ ]:
#Plot 2 dimension pdp for top 5 correlations
features_to_plot = ['price', 'bedrooms']
two_dim_pdp(f=features_to_plot)

Exlain result 3

In [ ]:
#Plot 2 dimension pdp for top 5 correlations
features_to_plot = ['price', 'bedrooms']
two_dim_pdp(f=features_to_plot)

Exlain result 4

In [ ]:
#Plot 2 dimension pdp for top 5 correlations
features_to_plot = ['price', 'bedrooms']
two_dim_pdp(f=features_to_plot)

Exlain result 5

Explain what shap values are

In [ ]:
shap_summary(rf, X_train)

Explain results

Explain what dependency contribution is

In [ ]:
dependency_contribution(f=top5_feat.iloc[0,0], i=0)

Explain result 1

In [ ]:
dependency_contribution(f=top5_feat.iloc[0,0], i=1)

Explain result 2

In [ ]:
dependency_contribution(f=top5_feat.iloc[0,0], i=2)

Explain result 3

In [ ]:
dependency_contribution(f=top5_feat.iloc[1,0], i=0)

Explain result 4

In [ ]:
dependency_contribution(f=top5_feat.iloc[1,0], i=1)

Explain result 5

In [ ]:
dependency_contribution(f=top5_feat.iloc[1,0], i=2)

Explain result 6

Explain what forceplots are

In [ ]:
n = np.random.randint(1,df.shape[0]+1)
m = np.random.randint(0,3)
forceplots(n, m)

Explain the results

## Extras

### Neural Network 

In [ ]:
n_classes = len(np.unique(y))
n_features = X_train.shape[1]

ffnn = Sequential()
ffnn.add(Dense(256, activation='relu', input_shape=(n_features,)))
ffnn.add(Dropout(0.3))
ffnn.add(Dense(64, activation='relu'))
ffnn.add(Dense(32, activation='relu'))
ffnn.add(Dense(n_classes, activation='softmax'))

ffnn.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=0.0001),
              metrics=['accuracy'])
ffnn.summary()

trained_ffnn = ffnn.fit(X_train, y_train,
          validation_data = (X_test, y_test),
          epochs=50, 
          batch_size=8, 
          verbose=1
         )

rcParams['figure.figsize'] = 10, 5
plt.plot(ffnn.history.history['loss'], label = "Train Loss")
plt.plot(ffnn.history.history['val_loss'], label = "Test Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend()
plt.plot()

rcParams['figure.figsize'] = 10, 5
plt.plot(ffnn.history.history['accuracy'], label = "Train Accuracy")
plt.plot(ffnn.history.history['val_accuracy'], label = "Test Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend()
plt.plot()